In [1]:
%reset -f

In [2]:
!pip install datasets rouge torch evaluate nltk tqdm --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 9.1 MB/s eta 0:00:00


In [3]:
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import datasets
import networkx
from rouge import Rouge
from evaluate import load
from nltk.translate.meteor_score import meteor_score
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import sys
sys.setrecursionlimit(1500)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [4]:
from datasets import load_dataset

dataset = load_dataset("ccdv/pubmed-summarization", split = 'test')
dataset = dataset.shuffle(seed=42).select(range(125))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for ccdv/pubmed-summarization contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ccdv/pubmed-summarization
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `d

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
art = []
abs = []

for dct in dataset:
    art.append(dct["article"])
    abs.append(dct["abstract"])

df_article, df_abstract = np.array(art), np.array(abs)

In [6]:
def normals(sent):
    stop_words = nltk.corpus.stopwords.words('english')

    sent = re.sub(r'[^a-zA-Z\s]', '', sent, re.I|re.A)
    sent = sent.lower()
    sent = sent.strip()

    # tokenize document
    tokens = nltk.word_tokenize(sent)

    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]

    # re-create document from filtered tokens
    sent = ' '.join(filtered_tokens)

    return sent

def text_tokens(doc):
    normals_all = np.vectorize(normals)

    doc = re.sub(r'\n|\r', ' ', doc)
    doc = re.sub(r' +', ' ', doc)
    doc = doc.strip()
    sentences = nltk.sent_tokenize(doc)
    num_sentence = int(np.ceil([len(sentences)/10]))

    sentences = normals_all(sentences)

    return sentences, num_sentence

def text_vectors(norm_sent):
    tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
    dt_matrix = tv.fit_transform(norm_sent)
    dt_matrix = dt_matrix.toarray()

    return dt_matrix

def sim_graph(vec_sent, visual=0):
    sim_matrix = np.matmul(vec_sent, vec_sent.T)
    similarity_graph = networkx.from_numpy_array(sim_matrix)

    if visual == 1:
        plt.figure(figsize=(12, 6))
        networkx.draw_networkx(similarity_graph, node_color='lime')

    return similarity_graph

def summary(graph, sentences, num_sentence):
    scores = networkx.pagerank(graph)
    ranked_sentences = sorted(((score, index) for index, score in scores.items()), reverse=True)
    top_sentence_indices = [ranked_sentences[index][1] for index in range(num_sentence)]
    top_sentence_indices.sort()

    return '\n'.join(np.array(sentences)[top_sentence_indices])

def TextRank(document, visual=0):
    token_texts, num_sentence = text_tokens(doc=document)
    token_vectors = text_vectors(norm_sent=token_texts)
    similarity_graph = sim_graph(vec_sent=token_vectors, visual=visual)
    text_summary = summary(graph=similarity_graph, sentences=token_texts, num_sentence=num_sentence)


    return text_summary

In [7]:
new_abstracts = []

for doc in tqdm(df_article):
    abstract = TextRank(document=doc)
    new_abstracts.append(abstract)

  0%|          | 0/125 [00:00<?, ?it/s]<ipython-input-6-6dd0c93598ce>:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  num_sentence = int(np.ceil([len(sentences)/10]))
100%|██████████| 125/125 [00:16<00:00,  7.49it/s]


In [8]:
fin = pd.concat([pd.DataFrame(df_article), pd.DataFrame(new_abstracts), pd.DataFrame(df_abstract)], axis=1)
fin.columns = ["original_article", "extractive_summary", "target_summary"]
fin.to_csv("Summarization_Results_125.csv", index=False)

In [12]:
rouge_scores = []
rouge = Rouge()

for row in np.array(fin):
    scores = rouge.get_scores(row[1], row[2])
    rouge_scores.append(scores[0])

In [13]:
rouge_1_r = 0
rouge_1_p = 0
rouge_1_f = 0
rouge_2_r = 0
rouge_2_p = 0
rouge_2_f = 0
rouge_l_r = 0
rouge_l_p = 0
rouge_l_f = 0

for row in rouge_scores:
    rouge_1_r += row["rouge-1"]["r"]/len(rouge_scores)
    rouge_1_p += row["rouge-1"]["p"]/len(rouge_scores)
    rouge_1_f += row["rouge-1"]["f"]/len(rouge_scores)
    rouge_2_r += row["rouge-2"]["r"]/len(rouge_scores)
    rouge_2_p += row["rouge-2"]["p"]/len(rouge_scores)
    rouge_2_f += row["rouge-2"]["f"]/len(rouge_scores)
    rouge_l_r += row["rouge-l"]["r"]/len(rouge_scores)
    rouge_l_p += row["rouge-l"]["p"]/len(rouge_scores)
    rouge_l_f += row["rouge-l"]["f"]/len(rouge_scores)

In [ ]:
#sentence by sentence comparison, not effective (must be the same size)
bleu_scores = []

for row in np.array(fin):
    scores = bleu_score([row[1]], [row[2]])
    bleu_scores.append(round(scores*100, 2))

In [ ]:
meteor_scores = []

for row in np.array(fin):
    score = meteor_score([nltk.word_tokenize(row[2])], nltk.word_tokenize(row[1]))
    meteor_scores.append(score)